# Customer Feedback Analysis

This notebook demonstrates how to analyze large-scale customer feedback using openaivec for sentiment analysis, categorization, and priority assignment.

In [ ]:
import pandas as pd
from openaivec import pandas_ext
from pydantic import BaseModel
from typing import Literal

# Setup
pandas_ext.responses_model("gpt-4o-mini")

## Sample Customer Feedback Data

Let's create realistic customer feedback data representing various scenarios.

In [ ]:
# Sample customer feedback data
feedback_data = [
    "The product quality is excellent, but delivery took 3 weeks. Very disappointed with shipping.",
    "Amazing customer service! Sarah helped me resolve my issue in 5 minutes. Will definitely buy again.",
    "App keeps crashing when I try to checkout. This is the third time this week. Fix your software!",
    "Love the new design updates. Much more user-friendly than before. Great job team!",
    "Pricing is too high compared to competitors. $50 for basic features is unreasonable.",
    "Website is slow and confusing. Took me 20 minutes to find what I was looking for.",
    "Perfect packaging, fast delivery, excellent product. 5 stars!",
    "Customer support never responds to emails. Been waiting 2 weeks for a refund.",
    "The mobile app crashes every time I try to upload photos. Please fix this bug.",
    "Great value for money. Been using this service for 2 years now, consistently good."
]

customer_df = pd.DataFrame({
    "customer_id": [f"CUST_{i:03d}" for i in range(1, len(feedback_data) + 1)],
    "feedback": feedback_data,
    "date": pd.date_range("2024-01-01", periods=len(feedback_data), freq="D")
})

customer_df.head()

## Structured Analysis

Define a structured output format for comprehensive feedback analysis.

In [ ]:
class FeedbackAnalysis(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"]
    category: Literal["product", "service", "delivery", "pricing", "technical", "website"]
    priority: Literal["low", "medium", "high", "urgent"]
    main_issue: str
    action_needed: str
    customer_emotion: Literal["satisfied", "frustrated", "angry", "delighted", "neutral"]

## AI-Powered Analysis

Process all feedback with structured analysis in a single operation.

In [ ]:
# Comprehensive feedback analysis
analyzed_df = customer_df.assign(
    analysis=lambda df: df.feedback.ai.responses(
        instructions="""
        Analyze the customer feedback and provide structured insights including:
        - Sentiment classification
        - Issue categorization  
        - Priority level for follow-up
        - Main issue summary
        - Recommended action
        - Customer emotional state
        """,
        response_format=FeedbackAnalysis
    )
).ai.extract("analysis")

analyzed_df.head()

## Business Intelligence Dashboard

Generate insights for business stakeholders.

In [ ]:
# Sentiment distribution
sentiment_summary = analyzed_df.analysis_sentiment.value_counts()
print("Sentiment Distribution:")
print(sentiment_summary)

print("\n" + "="*50 + "\n")

# Priority breakdown
priority_summary = analyzed_df.analysis_priority.value_counts()
print("Priority Breakdown:")
print(priority_summary)

print("\n" + "="*50 + "\n")

# Category insights
category_summary = analyzed_df.analysis_category.value_counts()
print("Issue Categories:")
print(category_summary)

## Action Items Generation

Create prioritized action items for different teams.

In [ ]:
# High priority items
urgent_issues = analyzed_df[
    analyzed_df.analysis_priority.isin(["high", "urgent"])
].sort_values("analysis_priority")

print("🚨 URGENT ACTION ITEMS:")
for _, row in urgent_issues.iterrows():
    print(f"\n📋 Customer: {row.customer_id}")
    print(f"   Category: {row.analysis_category.upper()}")
    print(f"   Issue: {row.analysis_main_issue}")
    print(f"   Action: {row.analysis_action_needed}")

## Conclusion

This example shows how openaivec can transform raw customer feedback into:
- Structured business intelligence
- Prioritized action items
- Team-specific insights
- Automated categorization

Scale this approach to thousands of reviews for comprehensive customer insight analysis.